In [21]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn
import torch.optim as optim
from torchvision.transforms import v2

In [22]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda


In [30]:
transforms = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToImage(),
    v2.ToDtype(torch.float32),
    v2.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05)
])

In [24]:
# Load the datasets
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=lambda y: torch.tensor(y, dtype=torch.long)
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=lambda y: torch.tensor(y, dtype=torch.long)
)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
class MyCNN (nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.convolutional_stack = nn.Sequential(
        nn.Conv2d(3, 8, 3),
        nn.BatchNorm2d(8),  # Batch normalization after convolution
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(8, 16, 3),
        nn.BatchNorm2d(16),  # Batch normalization after convolution
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Flatten(),
        nn.Linear(576, 120),
        nn.ReLU(),
        nn.Linear(120, 120),
        nn.ReLU(),
        nn.Linear(120,10)
         )
    def forward(self, x):
        logits = self.convolutional_stack(x)
        return logits

In [26]:
model = MyCNN().to(device)

train_loader = DataLoader(training_data, batch_size=4, shuffle=True)
test_loader = DataLoader(test_data, batch_size=4, shuffle=False)

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [27]:
for i in range(12):
    lossi = 0.0
    for input, label in train_loader:
        input, label = input.to(device), label.to(device)
        logits = model(input)
        loss = loss_fn(logits, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossi = loss.item()
    print(lossi)

0.9079392552375793
1.4606890678405762
0.3916834890842438
0.6289560794830322
1.1819095611572266
1.220747947692871
0.6954994201660156
0.6635934114456177
0.35209324955940247
1.2059999704360962
0.4059576690196991
0.1511380672454834


In [28]:
optimizer = optim.SGD(model.parameters(), lr = 0.001)
for i in range(2):
    lossi = 0.0

    for input, label in train_loader:
        input, label = input.to(device), label.to(device)
        logits = model(input)
        loss = loss_fn(logits, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossi = loss.item()
    print(lossi)

0.06777887791395187
0.23621954023838043


In [29]:
# Evaluation loop
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation during evaluation
    for input, label in test_loader:
        input, label = input.to(device), label.to(device)  # Move input and labels to device
        logits = model(input)
        _, predicted = torch.max(logits, 1)  # Get the predicted class with highest probability
        total += label.size(0)  # Increment the total number of samples
        correct += (predicted == label).sum().item()  # Increment the number of correctly predicted samples

print(f"Accuracy on the test set: {(correct / total) * 100:.2f}%")

Accuracy on the test set: 69.12%
